## Setup

In [1]:
!pip install langchain
!pip install gpt4all
!pip install qdrant-client
!pip install sentence-transformers
!pip install pypdf


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\rhihi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/812.8 kB ? eta -:--:--
     ------------------------------------- 812.8/812.8 kB 25.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/394.9 kB ? eta -:--:--
     ---------------------------------------- 394.9/394.9 kB ? eta 0:00:00
     ---------------------------------------- 0.0/276.8 kB ? eta -:--:--
     ------------------------------------- 276.8/276.8 kB 16.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/87.5 kB ? eta -:--:--
     ---------------------------------------- 87.5/87.5 kB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
     ---------------------------------------- 2.1/2.1 MB 43.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 1.9/1.9 MB 60.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/49.4 kB ? eta -:--:--
     ----------------


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\rhihi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/223.2 kB ? eta -:--:--
     ------------------------------------- 223.2/223.2 kB 13.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ---------------------------------------- 1.1/1.1 MB 34.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/121.1 kB ? eta -:--:--
     ---------------------------------------- 121.1/121.1 kB ? eta 0:00:00
     ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
     ---------------------------------------  3.8/3.8 MB 121.9 MB/s eta 0:00:01
     ---------------------------------------- 3.8/3.8 MB 61.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/75.6 kB ? eta -:--:--
     ---------------------------------------- 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/413.4 kB ? eta -:--:--
     ------------------------------------- 413.4/413.4 kB 25.2 MB/s eta 0:00:00
     ---------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyppeteer 2.0.0 requires urllib3<2.0.0,>=1.25.8, but you have urllib3 2.2.1 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\rhihi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/163.3 kB ? eta -:--:--
     ------------------------------------- 163.3/163.3 kB 10.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
     --------------- ------------------------ 3.3/8.8 MB 104.0 MB/s eta 0:00:01
     ------------------------------- -------- 6.9/8.8 MB 88.2 MB/s eta 0:00:01
     ---------------------------------------  8.8/8.8 MB 79.9 MB/s eta 0:00:01
     ---------------------------------------- 8.8/8.8 MB 56.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
     ----------- --------------------------- 3.2/10.6 MB 101.7 MB/s eta 0:00:01
     --------------------------------- ----- 9.2/10.6 MB 117.5 MB/s eta 0:00:01
     -------------------------------------  10.6/10.6 MB 108.8 MB/s eta 0:00:01
     --------------------------------------- 10.6/10.6 MB 65.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/388.9 kB ? eta -:


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\rhihi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/290.4 kB ? eta -:--:--
     ------------------------------------  286.7/290.4 kB 17.3 MB/s eta 0:00:01
     -------------------------------------- 290.4/290.4 kB 4.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\rhihi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Set huggingface cache home (optional)

In [1]:
import os
cache_dir="huggingface/"
os.environ["HF_HOME"] = cache_dir

model_dir = ".\\models\\mistral-7b-instruct-v0.1.Q4_0.gguf"
os.environ["MODEL"] = model_dir

## Check Device (assuming no GPU is available)

I have a GPU available but for you it will print "cpu" if you don't.

In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

## Enable to see Logs while running

In [3]:
import logging

# Configure logging
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Create a logger
logger = logging.getLogger(__name__)

## Load Data, preprocessing and Creating Qdrant Index

In [28]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import HuggingFaceEmbeddings
import re

def preprocess_text(text):
    """
    Preprocesses the text by normalizing it: making it lowercase, removing unnecessary
    punctuation while keeping essential characters for recipes.
    """
    text_lower = text.lower()
    # Remove unwanted characters but preserve those meaningful to recipes
    text_no_punctuation = re.sub(r'[^\w\s\$\%\.\,\"\-\:\'\!\?\(\)\n]', '', text_lower)
    text_normalized = re.sub(r'(\t)+', ' ', text_no_punctuation)  # Replace tabs with a single space
    return text_normalized

# Initialize the loader and the text splitter
loader = PyPDFLoader("data/Recipes1.pdf")
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator="\n")

# Load and preprocess documents
documents = loader.load()
preprocessed_documents = []

for document in documents:
    preprocessed_content = preprocess_text(document.page_content)
    # Here we check if the content fits within our desired chunk size
    if len(preprocessed_content) <= 1000:
        # If it fits, treat the whole recipe as a single chunk
        preprocessed_documents.append(preprocessed_content)
    else:
        # If it's too long, use the splitter to chunk it, although this shouldn't happen with typical recipes
        chunks = text_splitter.split_text(preprocessed_content)
        preprocessed_documents.extend(chunks)


docs = text_splitter.split_documents(documents)


embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5", 
                                   model_kwargs = {'device': "cpu"})  # forcefully setting device as cpu
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="msft_data",
    force_recreate=True
)

2024-04-10 10:57:41,441 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5
2024-04-10 10:57:41,559 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/modules.json HTTP/1.1" 200 0
2024-04-10 10:57:41,668 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/config_sentence_transformers.json HTTP/1.1" 200 0
2024-04-10 10:57:41,776 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/README.md HTTP/1.1" 200 0
2024-04-10 10:57:41,892 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/modules.json HTTP/1.1" 200 0
2024-04-10 10:57:42,006 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/sentence_bert_config.json HTTP/1.1" 200 0
2024-04-10 10:57:42,109 - urllib3.connecti

In [31]:
# Check if we have any preprocessed documents to print
if preprocessed_documents:
    # Print the first chunk/recipe to test
    print("First preprocessed chunk/recipe:")
    print(preprocessed_documents[56])
else:
    print("No preprocessed documents were found.")

First preprocessed chunk/recipe:
2 slices keto bread
75g turkey breast
tyson panko breaded pattie
20g tomatoshredded lettuce
1 sliced jalapeno pepper
2tbsp southwest hot mustard (optional)
60g fat free mozzarella
1tbsp light mayoi n g r e d i e n t sd.b.t.r. chicken turkey club
1.preheat the oven to 450 degrees and place one tyson panko-breaded breast in the oven. (20 minutes on one side, flip
and add 7 minutes) follow the directions on the bag to make sure it is
cooked thoroughly. use only half a breast on the sandwich.
2.slice up your tomato and fresh jalapeno. place your jalapeno in with the chicken and bake for
approximately 10 minutes. (check on it often pull out before chicken.)
3.toast your bread with spray butter on a medium heat stove or in the air fryer until desired
crispiness.
4.add 60 g of fat-free mozzarella to one or both sides of the bread and 75 g of deli turkey, and
microwave to melt or air fry 1-2 minutes at 400.


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import HuggingFaceEmbeddings
import re

def preprocess_text(text):
    text_lower = text.lower()
    text_no_punctuation = re.sub(r'[^\w\s\$\%\.\,\"\'\!\?\(\)]', '', text_lower)
    text_normalized_tabs = re.sub(r'(\t)+', '', text_no_punctuation)
    return text_normalized_tabs

# loader = PyPDFLoader("data/msft_annual_2023_report.pdf")
loader = PyPDFLoader("data/Recipes.pdf")
# loader = TextLoader(sample_texts)
# loader = WebBaseLoader("https://cleartax.in/s/top-performing-nps-schemes")
documents = loader.load()
for x in range(len(documents)):
    # do preprocessing
    documents[x].page_content=preprocess_text(documents[x].page_content)
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=500,separator="\n")
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5", 
                                   model_kwargs = {'device': "cpu"})  # forcefully setting device as cpu
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="msft_data",
    force_recreate=True
)

2024-04-10 10:28:36,759 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5
2024-04-10 10:28:36,761 - urllib3.connectionpool - DEBUG - Resetting dropped connection: huggingface.co
2024-04-10 10:28:37,024 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/modules.json HTTP/1.1" 200 0
2024-04-10 10:28:37,131 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/config_sentence_transformers.json HTTP/1.1" 200 0
2024-04-10 10:28:37,254 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/README.md HTTP/1.1" 200 0
2024-04-10 10:28:37,360 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/modules.json HTTP/1.1" 200 0
2024-04-10 10:28:37,463 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-e

## Run if not able to download model using GPT4All directly.

In [ ]:
# # download model

# !mkdir models
# !wget https://gpt4all.io/models/gguf/mistral-7b-instruct-v0.1.Q4_0.gguf -O models/mistral-7b-instruct-v0.1.Q4_0.gguf

## Directly calling GPT4All module 

In [5]:
from gpt4all import GPT4All
model = GPT4All(model_name="F:\RAG_PFE\RAG\models\mistral-7b-instruct-v0.1.Q4_0.gguf",                             n_threads = 8,
                             allow_download=False) # set allow_download as true to fetch it.

In [6]:
prompt = "Eiffel tower location:"
print(model.generate(prompt, max_tokens=20))

 Paris, France

Eiffel Tower is one of the most famous landmarks in the world


## Creating prompts, tuning parameters and creating Chain using Langchain

In [83]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, LLMChain
from langchain_community.llms import GPT4All

# template = '''[INST]: [\INST]\n
# Context: {context}.\n
# Question: {question}\n
# Answer: '''

# You are a chef with extensive knowledge of various recipes. 
# Based on the context provided, which could either be specific ingredients the 
# user has on hand or a type of dish the user desires, offer a suitable recipe that matches the request.

 
# As a chef renowned for your vast culinary knowledge and experience, use the context
# provided to deliver a comprehensive recipe. This entails not only guiding on the preparation
# of a specific dish with the available ingredients but also enriching the recipe with nutritional
# insights such as calorie count, protein, fat, and carbohydrate content. Whether the query
# involves utilizing specific ingredients at hand or crafting a dish to satisfy a particular
# craving, employ your expertise to recommend a balanced, nutritious recipe that caters to 
# the user's dietary preferences and goals. Include all necessary steps for preparation, 
# cooking tips, and garnishing advice to ensure a delightful culinary experience. Ensure 
# the recipe is clear, accessible, and tailored to the user's needs, providing a wholesome
# meal that is both satisfying and health-conscious.

template = '''[INST]:You are a chef expert bot, below presents a context from which the a question will be asked, give your valuable insights as well. and start by giving the name of the recipe, then the ingredientsm then the recipe steps, always. And always end the recipe , dont stop before writing all steps. [\INST]\n
Context: {context}.\n
Question: {question}\n
Answer: '''
rag_prompt = PromptTemplate(template=template, input_variables=["context","question"])

callbacks = [StreamingStdOutCallbackHandler()]

llm = GPT4All(
            model="F:\RAG_PFE\RAG\models\mistral-7b-instruct-v0.1.Q4_0.gguf",
            max_tokens=500,
            n_threads=10, 
            temp=0.3,
            top_p=0.2,
            top_k=40,
            n_batch=8,
            seed=100,
            allow_download=False,  # Prevent downloading
            verbose=True)

llm_chain = LLMChain(prompt=rag_prompt, llm=llm, verbose=True)

## Define format_docs for formatting context candidates.

In [68]:
def format_docs(query):
    found_docs = qdrant.similarity_search_with_score(query,k=1)
    return "\n\n".join(doc[0].page_content for doc in found_docs)

## Run Queries

In [71]:
%%time
query = "What can I make with chicken, tomatoes, and rice?"
context = "The user has chicken, tomatoes, and rice available and is looking for a recipe that uses these ingredients."
resp = llm_chain.invoke(
    input={"question": query,
           "context": context
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]:You are a chef expert bot, below presents a context from which the a question will be asked, give your valuable insights as well. and start by giving the name of the recipe, then the ingredientsm then the recipe steps, always. [\INST]

Context: The user has chicken, tomatoes, and rice available and is looking for a recipe that uses these ingredients..

Question: What can I make with chicken, tomatoes, and rice?

Answer: 


Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x000001FEE4ED7BE0>
Traceback (most recent call last):
  File "C:\Users\rhihi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gpt4all\_pyllmodel.py", line 526, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 



> Finished chain.
1) Chicken Fried Rice - This dish combines cooked rice, diced chicken, and sautéed tomatoes in a savory stir-fry. To make it, cook the rice according to package instructions, then sauté the chicken and tomatoes in oil until browned. Add the cooked rice to the pan and mix well. Serve hot with your favorite sauce.
2) Chicken Tomato Rice Pilaf - This dish is a flavorful one-pot meal that combines cooked rice, diced chicken, and sautéed tomatoes in a savory broth. To make it, cook the rice according to package instructions, then add the chicken and tomatoes to the pot with the rice. Add water or chicken broth and bring to a boil. Reduce heat and let simmer until the rice is cooked through and the liquid has been absorbed. Serve hot.
3) Chicken Tomato Stir-Fry - This dish is a quick and easy one that combines diced chicken, sautéed tomatoes, and stir-fried rice in oil. To make it, cook the rice according to package instructions, then sauté the chicken and tomatoes in oil 

In [72]:
%%time
query = "What can I make with chicken, tomatoes, and rice?"
resp = llm_chain.invoke(
    input={"question": query,
           "context": format_docs(query)
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]:You are a chef expert bot, below presents a context from which the a question will be asked, give your valuable insights as well. and start by giving the name of the recipe, then the ingredientsm then the recipe steps, always. [\INST]

Context: I n g r e d i e n t sAnabolic orange Chicken 
1.In a plastic ziplock bag, combine 6oz. Diced chicken with 2 TBSP corn starch Mix all of it together
by shaking the closed bag.
2.Heat a nonstick skillet to high heat with a 0 cal nonstick spray and add your chicken. Cook until
golden brown, turning the chicken over after 5–6 minutes. Make sure the chicken reaches an
internal temperature of 165°F or higher.
3.Once the chicken is cooked, add the 4 tablespoons of orange-ginger marinade over the chicken (in
the same pan), as well as the chili flakes and garlic.
4.Cook, stirring constantly, until the sauce and chicken are combined.
5.Serve over 1 cup of white rice and add your favorite g

## Other Queries ideas

query = 

In [63]:
%%time
query = "What can I make with chicken, tomatoes, and rice?"
resp = llm_chain.invoke(
    input={"question": query,
           "context": format_docs(query)
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]:You are a chef expert bot, below presents a context from which the a question will be asked, give your valuable insights as well. [\INST]

Context: I n g r e d i e n t sAnabolic orange Chicken 
1.In a plastic ziplock bag, combine 6oz. Diced chicken with 2 TBSP corn starch Mix all of it together
by shaking the closed bag.
2.Heat a nonstick skillet to high heat with a 0 cal nonstick spray and add your chicken. Cook until
golden brown, turning the chicken over after 5–6 minutes. Make sure the chicken reaches an
internal temperature of 165°F or higher.
3.Once the chicken is cooked, add the 4 tablespoons of orange-ginger marinade over the chicken (in
the same pan), as well as the chili flakes and garlic.
4.Cook, stirring constantly, until the sauce and chicken are combined.
5.Serve over 1 cup of white rice and add your favorite garnishes.D i r e c t i o n s560 calories 53 grams of protein
2 tbsp corn starch
4 TBSP orange gin

In [76]:
%%time
query = "What can I make with beef and potatoes?"
resp = llm_chain.invoke(
    input={"question": query,
           "context": format_docs(query)
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]:You are a chef expert bot, below presents a context from which the a question will be asked, give your valuable insights as well. and start by giving the name of the recipe, then the ingredientsm then the recipe steps, always. [\INST]

Context: I n g r e d i e n t sAnimal Fry Meal Prep 
First, let's make the fry sauce:
1.Combine 4 TBS of light mayo, 1 TBS of paprika, 3 TBS of sweet relish, 6 TBS of low-cal ketchup,
and 2 TBS of white vinegar.
Let's make the fries:
1.Place 340 grams of shoestring fries in the air fryer. 400 degrees for 15 minutes. Shake up the fries at
the 7-minute mark before continuing to cook.
2.Cook 20 g (93%) of beef with salt and pepper to taste. Add a few tablespoons of mustard, or my
favorite, dijon mustard, to the beef. Cook until thoroughly cooked on medium heat. I used a
nonstick pan.
3.Place the cooked beef off to the side in a separate container.
4.In the same pan, add 1 large white onion, c

In [74]:
%%time
query = "What can i make with eggs and flour?"
resp = llm_chain.invoke(
    input={"question": query,
           "context": format_docs(query)
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]:You are a chef expert bot, below presents a context from which the a question will be asked, give your valuable insights as well. and start by giving the name of the recipe, then the ingredientsm then the recipe steps, always. [\INST]

Context: 3 eggs
300g egg white 
3 cups simply potato hash browns 
1 cup mushrooms I n g r e d i e n t sMrs. Shreds Breakfast Bowl 
(3 servings)
 Set your oven to 425 and place 5 slices of center cut bacon onto a baking dish. Bake
your bacon for approximately 25 minutes on top rack or until desired crispiness.1.
 Get your favorite pan to medium heat and add 1 tbsp of olive oil or a zero cal
cooking spray.2.
 Add 3 cups of simply potato hash browns (feel free to cut this number in half if you
dont need the access calories)3.
 Cook for 6-7 mins on each side until golden brown. and set to the side. 4.
 Whisk 3 eggs and 300g of egg white and place into the same pan at medium heat.
Use salt and

In [77]:
%%time
query = "What can i make with apple, meat and rice?"
resp = llm_chain.invoke(
    input={"question": query,
           "context": format_docs(query)
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]:You are a chef expert bot, below presents a context from which the a question will be asked, give your valuable insights as well. and start by giving the name of the recipe, then the ingredientsm then the recipe steps, always. [\INST]

Context: 3 eggs
300g egg white 
3 cups simply potato hash browns 
1 cup mushrooms I n g r e d i e n t sMrs. Shreds Breakfast Bowl 
(3 servings)
 Set your oven to 425 and place 5 slices of center cut bacon onto a baking dish. Bake
your bacon for approximately 25 minutes on top rack or until desired crispiness.1.
 Get your favorite pan to medium heat and add 1 tbsp of olive oil or a zero cal
cooking spray.2.
 Add 3 cups of simply potato hash browns (feel free to cut this number in half if you
dont need the access calories)3.
 Cook for 6-7 mins on each side until golden brown. and set to the side. 4.
 Whisk 3 eggs and 300g of egg white and place into the same pan at medium heat.
Use salt and

In [84]:
%%time
query = "Give me a recipe for a chocolate cake."
resp = llm_chain.invoke(
    input={"question": query,
           "context": format_docs(query)
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]:You are a chef expert bot, below presents a context from which the a question will be asked, give your valuable insights as well. and start by giving the name of the recipe, then the ingredientsm then the recipe steps, always. And always end the recipe , dont stop before writing all steps. [\INST]

Context: I n g r e d i e n t sOreo Bday Cheesecake
1.In a blender, add 300 grams of nonfat Greek yogurt, 115 grams of reduced-fat cream cheese, 30 to
40 grams of a zero-calorie sweetener, 1 egg, 1 tablespoon of vanilla extract, and a splash of almond
milk. Blend until smooth.
2.Add contents to a small baking dish. I like putting down parchment paper with a little nonstick
spray.
3.Bake at 350 degrees for 30 minutes. After 30 minutes, take it out and let it cool until it's safe to place
in the freezer. I like the consistency after 2-3 hours covered in the freezer.
4.Frosting: 6 tbsp. cool whip lite; 10-15g stevia or monkfruit;

In [87]:
%%time
query = "What can I make with salmon, asparagus, and lemon?"
resp = llm_chain.invoke(
    input={"question": query,
           "context": format_docs(query)
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]:You are a chef expert bot, below presents a context from which the a question will be asked, give your valuable insights as well. and start by giving the name of the recipe, then the ingredientsm then the recipe steps, always. And always end the recipe , dont stop before writing all steps. [\INST]

Context: Add 2 pouches of Wild Caught
Smoked Salmon (Chicken of the
Sea) 
1 chopped celery stalk 
1/4 chopped red onion 
2 tbsp light mayoI n g r e d i e n t sdeviled egg salmon wrap (makes 2)
 step one, boil 2 eggs or more depending on how many you are making. 1.
 Once your eggs are boiled peel them and throw the egg shell away. 2.
 In a mixing bowl combine all above ingredients and mash together with a fork,
dont forget to hit it with some paprika and a dash of salt and pepper to taste.3.
 Split your deviled egg smoked salmon onto two different low cal tortillas. feel free
to add a slice of pepper jack, and about 30g of avo

In [86]:
%%time
query = "What can I make with chicken, tomatoes, and rice?"
resp = llm_chain.invoke(
    input={"question": query,
           "context": format_docs(query)
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]:You are a chef expert bot, below presents a context from which the a question will be asked, give your valuable insights as well. and start by giving the name of the recipe, then the ingredientsm then the recipe steps, always. And always end the recipe , dont stop before writing all steps. [\INST]

Context: I n g r e d i e n t sAnabolic orange Chicken 
1.In a plastic ziplock bag, combine 6oz. Diced chicken with 2 TBSP corn starch Mix all of it together
by shaking the closed bag.
2.Heat a nonstick skillet to high heat with a 0 cal nonstick spray and add your chicken. Cook until
golden brown, turning the chicken over after 5–6 minutes. Make sure the chicken reaches an
internal temperature of 165°F or higher.
3.Once the chicken is cooked, add the 4 tablespoons of orange-ginger marinade over the chicken (in
the same pan), as well as the chili flakes and garlic.
4.Cook, stirring constantly, until the sauce and chicken are co

In [88]:
%%time
query = "What can I make with shrimp, garlic, and spaghetti?"
resp = llm_chain.invoke(
    input={"question": query,
           "context": format_docs(query)
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]:You are a chef expert bot, below presents a context from which the a question will be asked, give your valuable insights as well. and start by giving the name of the recipe, then the ingredientsm then the recipe steps, always. And always end the recipe , dont stop before writing all steps. [\INST]

Context: I n g r e d i e n t sM ac N Cheese Pizza 
1.Measure out one serving of banza macaroni elbows from chickpeas. (2oz. dry)
2.Add 2 cups of water to a pot on high heat until the water reaches boiling.
3.Add your noodles and cook until the noodle softens. 7-8 min., stirring frequently.
4.In a blender or food processor, add 113 grams of fat-free or reduced-fat cottage cheese, salt, pepper,
1 tablespoon minced garlic, and a half packet of the cheese powder from the Banza box. Also, add
about 3 TBSP plain almond milk and blend until smooth.
5.Once the noodles are ready, drain and place them back into the original pot. reduce

In [90]:
%%time
query = "how can i make Chicken Alfredo Garlic Fries?"
resp = llm_chain.invoke(
    input={"question": query,
           "context": format_docs(query)
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]:You are a chef expert bot, below presents a context from which the a question will be asked, give your valuable insights as well. and start by giving the name of the recipe, then the ingredientsm then the recipe steps, always. And always end the recipe , dont stop before writing all steps. [\INST]

Context: 1.Preheat your oven or air fryer to 400 degrees.
2.Dice up a 4 oz. chicken breast into 6 thin pieces. season with salt, pepper, and garlic salt (your favorite
seasonings).
3.Put in the air fryer for 8 minutes (until cooked all the way through).
4.Place the fries in the air fryer, seasoned, for 4 minutes at 400 degrees.
5.Take the fries out and mix in parsley and garlic.
6.Throw the fries back in the air fryer for 4 more minutes (or until crispy).
7.Place one serving of alfredo sauce in a nonstick pan over medium heat on the stove.
8.Add 28 grams of fat-free cheddar and stir until melted.
9.Plate your fries or chicken

In [91]:
%%time
query = "how can i make egg, flour and milk?"
resp = llm_chain.invoke(
    input={"question": query,
           "context": format_docs(query)
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]:You are a chef expert bot, below presents a context from which the a question will be asked, give your valuable insights as well. and start by giving the name of the recipe, then the ingredientsm then the recipe steps, always. And always end the recipe , dont stop before writing all steps. [\INST]

Context: 3 eggs
300g egg white 
3 cups simply potato hash browns 
1 cup mushrooms I n g r e d i e n t sMrs. Shreds Breakfast Bowl 
(3 servings)
 Set your oven to 425 and place 5 slices of center cut bacon onto a baking dish. Bake
your bacon for approximately 25 minutes on top rack or until desired crispiness.1.
 Get your favorite pan to medium heat and add 1 tbsp of olive oil or a zero cal
cooking spray.2.
 Add 3 cups of simply potato hash browns (feel free to cut this number in half if you
dont need the access calories)3.
 Cook for 6-7 mins on each side until golden brown. and set to the side. 4.
 Whisk 3 eggs and 300g of eg